## ちょっと思いついたことを試すファイル --
* もしかして0.01ノイズ入ってるとかしない？ --

In [239]:
import warnings; warnings.simplefilter("ignore")

import os, sys, gc, random
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

from config import *

### データ読み込み

In [240]:
df = pd.read_feather("./input/df_feature01.feather")

In [241]:
train = df[df["train_test"] == "train"]
test = df[df["train_test"] == "test"]

train = train.drop(["train_test", "index", "Gender_normal", "disease_normal"], axis=1)
test = test.drop(["train_test", "index", "disease", "Gender_normal", "disease_normal"], axis=1).reset_index(drop=True)

In [242]:
train["disease"] = train["disease"].astype(int)

In [243]:
# train = pd.read_csv(f"{args.data}train.csv")
# test = pd.read_csv(f"{args.data}test.csv")
sample_submission = pd.read_csv(f"{args.data}sample_submit.csv")
print(train.shape, test.shape, sample_submission.shape)

(850, 383) (350, 382) (349, 2)


## oofをread

In [244]:
oof = pd.read_csv("./output/Lgb_Baseline/oof.csv")
sub = pd.read_csv("./output/Lgb_Baseline/submission.csv")

In [245]:
train["lgb_oof"] = oof["disease"]
test["lgb_oof"] = sub["0.1"]

### ノイズ除去?

In [246]:
## cols = [col for col in train.columns if col not in ["Age", "Gender", "disease"]]
## 
## for col in cols:
##     train[col] = train[col].map(lambda x: np.round(x, 3))
##     test[col] = test[col].map(lambda x: np.round(x, 3))

## Series Featureでベースライン

### Categoryを変換

In [247]:
train["Gender"] = train["Gender"].map(lambda x: {"Male":0, "Female":1}[x])
test["Gender"] = test["Gender"].map(lambda x: {"Male":0, "Female":1}[x])

### ベースライン作成

In [248]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

folds = 5
seed = 42
run_id = "Trial_05"

output_path = "./output/" + run_id

if not os.path.exists(output_path):
    os.mkdir(output_path)

In [249]:
from sklearn.metrics import log_loss, roc_auc_score
def Metric(labels, preds):
    return roc_auc_score(labels, preds)

In [250]:
cat_feature = []
num_feature = [col for col in train.columns if col not in ["disease"]]
features = cat_feature + num_feature
label_name = "disease"

In [251]:
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
split = skf.split(train, train[label_name])

oof = pd.DataFrame(train.index, columns=["index"])
oof[label_name] = 0

In [252]:
lgb_params = {
    "objective": "binary",
    "metric": "binary_logloss",

    "max_depth": -1,
    "num_leaves": 64,
    "learning_rate": 0.035,
    "bagging_freq": 5,
    "bagging_fraction": 0.7,
    "feature_fraction": 0.7,

    "min_data_in_leaf": 44,
    "lambda_l1": 0.1,
    #"lambda_l2": 30,

    "num_threads": 16,
}

In [253]:
all_valid_metric, feature_importance = [], []
for fold, (trn_index, val_index) in enumerate(split):
    evals_result_dic = {}
    trn_data = lgb.Dataset(train.loc[trn_index, features], label=train.loc[trn_index, label_name])
    val_data = lgb.Dataset(train.loc[val_index, features], label=train.loc[val_index, label_name])

    model = lgb.train(lgb_params,
        train_set = trn_data,
        num_boost_round = 10000,
        valid_sets = [trn_data, val_data],
        evals_result = evals_result_dic,
        early_stopping_rounds = 2000,
        verbose_eval = -1
    )
    model.save_model(output_path + "/fold%s.ckpt"%fold)

    valid_preds = model.predict(train.loc[val_index, features], num_iterations=model.best_iteration)
    oof.loc[val_index, label_name] = valid_preds

    all_valid_metric.append(Metric(train.loc[val_index, label_name], valid_preds))

    importance_gain = model.feature_importance(importance_type="gain")
    importance_split = model.feature_importance(importance_type="split")
    feature_name = model.feature_name()
    feature_importance.append(
        pd.DataFrame(
            {"feature_name": feature_name,
             "importance_gain": importance_gain,
             "importance_split": importance_split})
    )
feature_importance_df = pd.concat(feature_importance)
feature_importance_df = feature_importance_df.groupby(["feature_name"]).mean().reset_index()
feature_importance_df = feature_importance_df.sort_values(by=["importance_gain"], ascending=False)
feature_importance_df.to_csv(output_path + "/feature_importance.csv", index=False)

global_valid_metric = Metric(train[label_name].values, oof[label_name].values)
oof.to_csv(output_path + "/oof.csv", index=False)


[LightGBM] [Info] Number of positive: 303, number of negative: 377
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005445 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56019
[LightGBM] [Info] Number of data points in the train set: 680, number of used features: 383
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.445588 -> initscore=-0.218512
[LightGBM] [Info] Start training from score -0.218512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 2000 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

In [254]:
print("mean valid metric", np.mean(all_valid_metric))
print("global valid metric", global_valid_metric)

mean valid metric 0.9373016050765213
global valid metric 0.9362781364900009


In [255]:
sub = pd.DataFrame(test.index, columns=["index"])
sub["prediction"] = 0
for fold in range(folds):
    model = lgb.Booster(model_file = output_path + "/fold%s.ckpt"%fold)
    test_preds = model.predict(test[features], num_iteration=model.best_iteration)
    sub["prediction"] += (test_preds / folds)

In [256]:
sub.rename(columns={"index":0, "prediction":0.1}, inplace=True)

In [257]:
sub.to_csv(output_path + "/submission.csv", index=False)

In [258]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(train.loc[val_index, label_name].values, valid_preds))

0.9431690929451286
